# <center> Finding abnormal Users </center>

In [ ]:
import tensorflow as tf  #This idea did not work
import pandas as pd
import numpy as np
import pickle
from collections import defaultdict

## Importing events dataset

In [37]:
# Reading event datasets 
events = pd.read_csv("../data/events.csv")
events.head()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


In [38]:
# Describing quantitative features of events dataset
print ("No. of rows in events dataset : {0}".format(len(events)))
events.describe()

No. of rows in events dataset : 2756101


,timestamp,visitorid,itemid,transactionid
count,2.756101e+06,2.756101e+06,2.756101e+06,22457.000000
mean,1.436424e+12,7.019229e+05,2.349225e+05,8826.497796
std,3.366312e+09,4.056875e+05,1.341954e+05,5098.996290
min,1.430622e+12,0.000000e+00,3.000000e+00,0.000000
25%,1.433478e+12,3.505660e+05,1.181200e+05,4411.000000
50%,1.436453e+12,7.020600e+05,2.360670e+05,8813.000000
75%,1.439225e+12,1.053437e+06,3.507150e+05,13224.000000
max,1.442545e+12,1.407579e+06,4.668670e+05,17671.000000


In [39]:
# statistics for categorical variables
events.describe(include=[np.object])

,event
count,2756101
unique,3
top,view
freq,2664312


## Build userDict from events DF

In [40]:
def getDictOfUser(events):
    """
    Given   : events as pandas dataframe
    Return  : dictionary of all users- This can be treated as userProfiles
    """
    userDict = defaultdict(lambda : defaultdict(list))
    for index, row in events.iterrows():
        if row['event']=='view':
            userDict[row['visitorid']]['view'].append(row['itemid'])
        if row['event']=='addtocart':
            userDict[row['visitorid']]['addtocart'].append(row['itemid'])
            
    return userDict.copy()

userDict = getDictOfUser(events)

NameError: name 'defaultdict' is not defined

In [36]:
# Loading userProfile custom built from the dataset provided
userDF = pd.read_pickle("../output/userDF.pickle")
userDF.head()

,userid,view1,view2,view3,view4,view5,view6,addtocart
0,786432.0,58086.0,325585.0,58086.0,325585.0,0.0,0.0,325585.0
1,917508.0,218033.0,218033.0,22839.0,218033.0,0.0,0.0,218033.0
2,6.0,344723.0,344723.0,253615.0,344723.0,344723.0,0.0,65273.0
3,1310734.0,390399.0,443094.0,0.0,0.0,0.0,0.0,435940.0
4,567388.0,340375.0,340375.0,80435.0,80435.0,88904.0,80435.0,88904.0


In [3]:
# This give us the no. of unique items
print ("The no. of unique items in dataset : {0}".format(len(userDF.addtocart.unique())))
len(userDF.addtocart.unique())

The no. of unique items in dataset : 23208


23208

In [4]:
len(userDF)

97169

## Building Model

In [5]:
def one_hot_matrix(labels, C):
    """
    Creates a matrix where the i-th row corresponds to the ith class number and the jth column
                     corresponds to the jth training example. So if example j had a label i. Then entry (i,j) 
                     will be 1. 
                     
    Arguments:
    labels -- vector containing the labels 
    C -- number of classes, the depth of the one hot dimension
    
    Returns: 
    one_hot -- one hot matrix
    """
    
       
    # Create a tf.constant equal to C (depth), name it 'C'. (approx. 1 line)
    C = tf.constant(value=C, name='C')
    one_hot_matrix = tf.one_hot(indices = labels, depth = C, on_value=1, off_value=0, axis=0)
    sess = tf.Session()
    one_hot = sess.run(one_hot_matrix)
    sess.close()
    
    return one_hot

In [6]:
userDF.head()
len(userDF.addtocart.unique())

23208

In [7]:
# userDF = userDF.drop('userid',axis=1)
userDF2 = userDF.as_matrix()
userDF2 = userDF2.reshape(userDF2.shape[0], -1).T
# userDF2
len(userDF.addtocart.unique())

23208

In [26]:
k =round(userDF2.shape[1]*.9)
train, test = userDF2[:,:k], userDF2[:,k:]
X_train, Y_train = train[:6,:], train[6:,:].astype(np.int64)
X_test, Y_test = test[:6,:], test[6:, :].astype(np.int64)

# Convert training and test labels to one hot matrices

def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y


# Y_train = convert_to_one_hot(Y_train, 100)
# Y_test = convert_to_one_hot(Y_test, 100)

print ("number of training examples = " + str(X_train.shape[1]))
print ("number of test examples = " + str(X_test.shape[1]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 87452
number of test examples = 9717
X_train shape: (6, 87452)
Y_train shape: (2, 87452)
X_test shape: (6, 9717)
Y_test shape: (2, 9717)


In [9]:
def create_placeholders(n_x, n_y):
    """
    Creates the placeholders for the tensorflow session.
    
    Arguments:
    n_x -- scalar, size of an image vector (num_px * num_px = 64 * 64 * 3 = 12288)
    n_y -- scalar, number of classes (from 0 to 5, so -> 6)
    
    Returns:
    X -- placeholder for the data input, of shape [n_x, None] and dtype "float"
    Y -- placeholder for the input labels, of shape [n_y, None] and dtype "float"
    
    Tips:
    - You will use None because it let's us be flexible on the number of examples you will for the placeholders.
      In fact, the number of examples during test/train is different.
    """
    
    X = tf.placeholder(dtype=tf.float32, shape=[n_x,None])
    Y = tf.placeholder(dtype=tf.float32, shape=[n_y,None])
    
    return X, Y

**The model** is *LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SOFTMAX*. The SIGMOID output layer has been converted to a SOFTMAX. A SOFTMAX layer generalizes SIGMOID to when there are more than two classes. 

In [10]:
def initialize_parameters():
    """
    Initializes parameters to build a neural network with tensorflow. The shapes are:
                        W1 : [25, 6]
                        b1 : [25, 1]
                        W2 : [12, 25]
                        b2 : [12, 1]
                        W3 : [6, 12]
                        b3 : [6, 1]
    
    Returns:
    parameters -- a dictionary of tensors containing W1, b1, W2, b2, W3, b3
    """
    
    tf.set_random_seed(1)                   # so that your "random" numbers match ours
        
    ### START CODE HERE ### (approx. 6 lines of code)
    W1 = tf.get_variable(name="W1", shape=[25,6], initializer=tf.contrib.layers.xavier_initializer(seed = 1))
    b1 = tf.get_variable(name="b1", shape = [25,1], initializer=tf.zeros_initializer())
    W2 = tf.get_variable(name="W2", shape=[12,25], initializer=tf.contrib.layers.xavier_initializer(seed = 1))
    b2 = tf.get_variable(name="b2", shape = [12,1], initializer=tf.zeros_initializer())
    W3 = tf.get_variable(name="W3", shape=[6,12], initializer=tf.contrib.layers.xavier_initializer(seed = 1))
    b3 = tf.get_variable(name="b3", shape = [6,1], initializer=tf.zeros_initializer())
    ### END CODE HERE ###

    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2,
                  "W3": W3,
                  "b3": b3}
    
    return parameters

In [35]:
convert_to_one_hot(np.array(Y_test),Y_test.max()+1)

MemoryError: 

In [33]:
Y_test.max()

466864